In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("/Users/sckakani/Workspace/Classwork/cs5804/miniproject/I-66E_to_Glebe.csv", index_col = 0)


In [3]:
df = df.drop(index = df.query("Hour_Volume == '000-1'").index)
df = df.astype({'Hour_Volume':'int'})
df['DateTime'] = df.apply(lambda x : datetime(x['Year_Record'] + 2000, x['Month_Record'], x['Day_Record'], x['Hour_Record']), axis = 1)
df = df.set_index('DateTime')

In [4]:
df.head()

,Record_Type,State_Code,F_System,Station_Id,Travel_Dir,Travel_Lane,Year_Record,Month_Record,Day_Record,Day_of_Week,Hour_Record,Hour_Volume,Restrictions
DateTime,,,,,,,,,,,,,
2011-01-01 00:00:00,3,51,1U,90106,3,0,11,1,1,7,0,799,0
2011-01-01 01:00:00,3,51,1U,90106,3,0,11,1,1,7,1,1097,0
2011-01-01 02:00:00,3,51,1U,90106,3,0,11,1,1,7,2,762,0
2011-01-01 03:00:00,3,51,1U,90106,3,0,11,1,1,7,3,465,0
2011-01-01 04:00:00,3,51,1U,90106,3,0,11,1,1,7,4,342,0


In [5]:
# X = df.drop(['Record_Type', 'State_Code', 'F_System', 'Station_Id', 'Travel_Dir', 'Travel_Lane', 'Restrictions', 'Hour_Volume'], axis=1)

# y = df['Hour_Volume']
training = df.query("Year_Record < 22")
testing = df.query("Year_Record >= 22")

In [6]:
from sklearn.model_selection import train_test_split
X_train = training.drop(['Record_Type', 'State_Code', 'F_System', 'Station_Id', 'Travel_Dir', 'Travel_Lane', 'Restrictions', 'Hour_Volume'], axis=1)

y_train = training['Hour_Volume']

X_test = testing.drop(['Record_Type', 'State_Code', 'F_System', 'Station_Id', 'Travel_Dir', 'Travel_Lane', 'Restrictions', 'Hour_Volume'], axis=1)

y_test = testing['Hour_Volume']

In [7]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators=100)

In [8]:
rfr.fit(X_train, y_train)

RandomForestRegressor()

In [9]:
y_pred = rfr.predict(X_train)
from sklearn.metrics import mean_squared_error
print( math.sqrt(mean_squared_error(y_train, y_pred)))

72.59060788126261


In [10]:
y_pred = rfr.predict(X_test)
from sklearn.metrics import mean_squared_error
print( math.sqrt(mean_squared_error(y_test, y_pred)))

441.7955750586375


In [11]:
print(rfr.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [12]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10, 20], 'min_samples_leaf': [1, 2, 4, 8], 'bootstrap': [True, False]}


In [13]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/Users/sckakani/miniconda3/envs/P4Data/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/sckakani/miniconda3/envs/P4Data/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/sckakani/miniconda3/envs/P4Data/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0`

[CV] END bootstrap=True, max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   3.7s
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=500; total time=  27.5s


/Users/sckakani/miniconda3/envs/P4Data/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/sckakani/miniconda3/envs/P4Data/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


[CV] END bootstrap=True, max_depth=110, max_features=log2, min_samples_leaf=2, min_samples_split=20, n_estimators=200; total time=   6.8s
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=500; total time=  27.6s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=20, n_estimators=100; total time=   7.4s
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=500; total time=  27.4s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=20, n_estimators=100; total time=   7.5s
[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=8, min_samples_split=5, n_estimators=500; total time=  16.5s
[CV] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=8, min_samples_split=10, n_estimators=700; total time=  14.9s
[CV] END bootstrap=True, max_de

...

[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=8, min_samples_split=10, n_estimators=400; total time=  43.5s
[CV] END bootstrap=True, max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=20, n_estimators=900; total time=  28.5s
[CV] END bootstrap=True, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=800; total time= 1.1min


 or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/sckakani/miniconda3/envs/P4Data/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/sckakani/miniconda3/envs/P4Data/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/sckakani/miniconda3/envs/P4Data/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto

[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=800; total time= 1.8min
[CV] END bootstrap=True, max_depth=40, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=900; total time= 1.2min
[CV] END bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=800; total time= 1.6min
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=8, min_samples_split=20, n_estimators=400; total time=  18.7s
[CV] END bootstrap=False, max_depth=10, max_features=log2, min_samples_leaf=8, min_samples_split=20, n_estimators=600; total time=  19.0s
[CV] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=20, n_estimators=100; total time=  11.4s
[CV] END bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=900; total time=  48.4s
[CV] END bootstrap=True, max_depth=

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 8],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
                   random_state=42, verbose=2)

[CV] END bootstrap=True, max_depth=100, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=900; total time=  36.8s
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=500; total time= 1.1min
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=20, n_estimators=900; total time=  44.6s
[CV] END bootstrap=False, max_depth=100, max_features=log2, min_samples_leaf=8, min_samples_split=20, n_estimators=600; total time=  29.6s
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=4, min_samples_split=20, n_estimators=400; total time=  20.6s
[CV] END bootstrap=False, max_depth=110, max_features=log2, min_samples_leaf=8, min_samples_split=5, n_estimators=700; total time=  34.8s
[CV] END bootstrap=False, max_depth=80, max_features=log2, min_samples_leaf=4, min_samples_split=20, n_estimators=600; total time=  28.2s
[CV] END bootstrap=False, max_depth

In [14]:
rf_random.best_params_

{'n_estimators': 900,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'max_depth': 100,
 'bootstrap': True}